In [311]:
import pandas as pd

In [312]:
import numpy as np

In [313]:
crop_df = pd.read_csv(r'D:\ECO221_Group-25\data\ECO221_Project_2025_Final.csv')

In [314]:
rainfall_df = pd.read_csv(r'D:\ECO221_Group-25\data\RF_DistrictWise_ECO221_2025.csv')

In [315]:
salinity_df = pd.read_csv(r'D:\ECO221_Group-25\data\Salinity_Alkalinity_ECO221_2025.csv')

In [316]:
sugarcane_df = crop_df[crop_df['crop'].str.lower() == 'sugarcane'].copy()

In [317]:
sugarcane_df.isnull().sum()

crop                           0
area1000hectares              65
production1000tonnes          65
irrigatedarea1000hectares     65
districtcode                   0
year                           0
statecode                      0
statename                      0
nitrogenconsumptiontonnes      0
phosphateconsumptiontonnes     0
potashconsumptiontonnes        0
districtname                   0
dtype: int64

In [318]:
sugarcane_df.dropna(inplace = True)

In [319]:
sugarcane_df

,crop,area1000hectares,production1000tonnes,irrigatedarea1000hectares,districtcode,year,statecode,statename,nitrogenconsumptiontonnes,phosphateconsumptiontonnes,potashconsumptiontonnes,districtname
8,Sugarcane,19.85,3.59,19.820,1,2017,14,Chhattisgarh,91447,53486,16993,Durg
35,Sugarcane,0.72,0.11,0.616,2,2017,14,Chhattisgarh,21279,12584,4271,Bastar
64,Sugarcane,0.27,0.05,0.267,3,2017,14,Chhattisgarh,92008,47721,15054,Raipur
71,Sugarcane,7.57,0.83,7.484,4,2017,14,Chhattisgarh,72926,29713,6915,Bilaspur
95,Sugarcane,0.43,0.09,0.431,5,2017,14,Chhattisgarh,31282,13620,3707,Raigarh
...,...,...,...,...,...,...,...,...,...,...,...,...
5630,Sugarcane,0.23,1.42,0.120,907,2017,2,Bihar,90899,24771,4278,Patna
5635,Sugarcane,2.53,15.81,0.490,908,2017,2,Bihar,125661,42562,21130,Mungair
5670,Sugarcane,2.63,14.79,0.820,909,2017,2,Bihar,60780,19306,11211,Bhagalpur
5717,Sugarcane,0.24,1.53,0.230,911,2017,2,Bihar,162928,50799,7932,Shahabad (now part of Bhojpur district)


In [320]:
#Capping the irrigated Area
sugarcane_df['irrigatedarea_capped'] = sugarcane_df[["irrigatedarea1000hectares", "area1000hectares"]].min(axis=1)

In [321]:
# sugarcane_df['irrigatedarea_new'] = sugarcane_df['irrigatedarea_capped'] + 1

In [322]:
# sugarcane_df['unirrigated_area'] = sugarcane_df['area1000hectares'] - sugarcane_df['irrigatedarea_capped']


In [323]:
initial_districts = crop_df[crop_df["crop"].str.lower() == "sugarcane"]["districtname"].str.lower().str.strip().nunique()
print("Initial sugarcane districts:", initial_districts)

Initial sugarcane districts: 266


In [324]:
rainfall_2017 = rainfall_df[rainfall_df['yearcode'] == 2017].copy()
rainfall_2017['month'] = rainfall_2017['month'].str.lower().str.strip()
rainfall_2017['district'] = rainfall_2017['district'].str.lower().str.strip()

In [325]:
monsoon_months = ['june, 2017', 'july, 2017', 'august, 2017', 'september, 2017']


In [326]:
# === Aggregate annual rainfall ===
annual_rainfall = rainfall_2017.groupby('district')['rf'].sum().reset_index()
annual_rainfall.columns = ['districtname', 'annual_rainfall_mm']

In [327]:
# === Aggregate monsoon rainfall ===
monsoon_rainfall = rainfall_2017[rainfall_2017['month'].isin(monsoon_months)]
monsoon_summary = monsoon_rainfall.groupby('district')['rf'].sum().reset_index()
monsoon_summary.columns = ['districtname', 'monsoon_rainfall_mm']


In [328]:
sugarcane_df['districtname'] = sugarcane_df['districtname'].str.lower().str.strip()
annual_rainfall['districtname'] = annual_rainfall['districtname'].str.lower().str.strip()
monsoon_summary['districtname'] = monsoon_summary['districtname'].str.lower().str.strip()

In [329]:
# === Merge rainfall summaries ===
sugarcane_df = pd.merge(sugarcane_df, annual_rainfall, on='districtname', how='inner')
sugarcane_df = pd.merge(sugarcane_df, monsoon_summary, on='districtname', how='inner')
# sugarcane_df = pd.merge(sugarcane_df, annual_rainfall, on='districtname', how='left')
# sugarcane_df = pd.merge(sugarcane_df, monsoon_summary, on='districtname', how='left')

In [330]:
sugarcane_df

,crop,area1000hectares,production1000tonnes,irrigatedarea1000hectares,districtcode,year,statecode,statename,nitrogenconsumptiontonnes,phosphateconsumptiontonnes,potashconsumptiontonnes,districtname,irrigatedarea_capped,annual_rainfall_mm,monsoon_rainfall_mm
0,Sugarcane,19.850000,3.59,19.820000,1,2017,14,Chhattisgarh,91447,53486,16993,durg,19.820000,861.629998,805.980000
1,Sugarcane,0.720000,0.11,0.616000,2,2017,14,Chhattisgarh,21279,12584,4271,bastar,0.616000,902.189990,781.249990
2,Sugarcane,0.270000,0.05,0.267000,3,2017,14,Chhattisgarh,92008,47721,15054,raipur,0.267000,683.989988,621.329990
3,Sugarcane,7.570000,0.83,7.484000,4,2017,14,Chhattisgarh,72926,29713,6915,bilaspur,7.484000,1857.730027,1539.510022
4,Sugarcane,0.430000,0.09,0.431000,5,2017,14,Chhattisgarh,31282,13620,3707,raigarh,0.430000,624.520000,563.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Sugarcane,0.150000,0.87,0.130000,904,2017,2,Bihar,44082,18294,11692,saharsa,0.130000,1101.630021,839.480022
165,Sugarcane,27.690001,158.70,19.629999,906,2017,2,Bihar,65685,16211,3535,saran,19.629999,529.989991,359.050002
166,Sugarcane,0.230000,1.42,0.120000,907,2017,2,Bihar,90899,24771,4278,patna,0.120000,700.579998,504.499998
167,Sugarcane,2.630000,14.79,0.820000,909,2017,2,Bihar,60780,19306,11211,bhagalpur,0.820000,958.229988,625.799990


In [331]:
print("Merged sugarcane data with rainfall:")
print(sugarcane_df[['districtname', 'annual_rainfall_mm', 'monsoon_rainfall_mm']].head(10))

Merged sugarcane data with rainfall:
  districtname  annual_rainfall_mm  monsoon_rainfall_mm
0         durg          861.629998           805.980000
1       bastar          902.189990           781.249990
2       raipur          683.989988           621.329990
3     bilaspur         1857.730027          1539.510022
4      raigarh          624.520000           563.340000
5      surguja          580.889988           516.769989
6     jabalpur          826.810001           775.400000
7     balaghat          962.159988           899.959990
8   chhindwara          953.030709           882.499790
9  narsinghpur          817.670000           771.730000


In [332]:
merged_districts_task1 = sugarcane_df['districtname'].nunique()
missing_rain = sugarcane_df[sugarcane_df['annual_rainfall_mm'].isna() | sugarcane_df['monsoon_rainfall_mm'].isna()]['districtname'].unique()
print("Merged districts (Task 1):", merged_districts_task1)
print("Districts missing rainfall data:", len(missing_rain))

Merged districts (Task 1): 169
Districts missing rainfall data: 0


In [333]:
sugarcane_df.isnull().sum()

crop                          0
area1000hectares              0
production1000tonnes          0
irrigatedarea1000hectares     0
districtcode                  0
year                          0
statecode                     0
statename                     0
nitrogenconsumptiontonnes     0
phosphateconsumptiontonnes    0
potashconsumptiontonnes       0
districtname                  0
irrigatedarea_capped          0
annual_rainfall_mm            0
monsoon_rainfall_mm           0
dtype: int64

In [334]:
sugarcane_df.dropna(inplace = True)

In [335]:
sugarcane_df

,crop,area1000hectares,production1000tonnes,irrigatedarea1000hectares,districtcode,year,statecode,statename,nitrogenconsumptiontonnes,phosphateconsumptiontonnes,potashconsumptiontonnes,districtname,irrigatedarea_capped,annual_rainfall_mm,monsoon_rainfall_mm
0,Sugarcane,19.850000,3.59,19.820000,1,2017,14,Chhattisgarh,91447,53486,16993,durg,19.820000,861.629998,805.980000
1,Sugarcane,0.720000,0.11,0.616000,2,2017,14,Chhattisgarh,21279,12584,4271,bastar,0.616000,902.189990,781.249990
2,Sugarcane,0.270000,0.05,0.267000,3,2017,14,Chhattisgarh,92008,47721,15054,raipur,0.267000,683.989988,621.329990
3,Sugarcane,7.570000,0.83,7.484000,4,2017,14,Chhattisgarh,72926,29713,6915,bilaspur,7.484000,1857.730027,1539.510022
4,Sugarcane,0.430000,0.09,0.431000,5,2017,14,Chhattisgarh,31282,13620,3707,raigarh,0.430000,624.520000,563.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Sugarcane,0.150000,0.87,0.130000,904,2017,2,Bihar,44082,18294,11692,saharsa,0.130000,1101.630021,839.480022
165,Sugarcane,27.690001,158.70,19.629999,906,2017,2,Bihar,65685,16211,3535,saran,19.629999,529.989991,359.050002
166,Sugarcane,0.230000,1.42,0.120000,907,2017,2,Bihar,90899,24771,4278,patna,0.120000,700.579998,504.499998
167,Sugarcane,2.630000,14.79,0.820000,909,2017,2,Bihar,60780,19306,11211,bhagalpur,0.820000,958.229988,625.799990


In [336]:
sugarcane_df.isnull().sum()

crop                          0
area1000hectares              0
production1000tonnes          0
irrigatedarea1000hectares     0
districtcode                  0
year                          0
statecode                     0
statename                     0
nitrogenconsumptiontonnes     0
phosphateconsumptiontonnes    0
potashconsumptiontonnes       0
districtname                  0
irrigatedarea_capped          0
annual_rainfall_mm            0
monsoon_rainfall_mm           0
dtype: int64

In [337]:
sugarcane_df.to_csv('D:\ECO221_Group-25\data\ task1.csv', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\DELL\AppData\Local\Temp\ipykernel_20032\4013827872.py:1: SyntaxWarning: invalid escape sequence '\E'
  sugarcane_df.to_csv('D:\ECO221_Group-25\data\ task1.csv', index=False)
C:\Users\DELL\AppData\Local\Temp\ipykernel_20032\4013827872.py:1: SyntaxWarning: invalid escape sequence '\E'
  sugarcane_df.to_csv('D:\ECO221_Group-25\data\ task1.csv', index=False)


PermissionError: [Errno 13] Permission denied: 'D:\\ECO221_Group-25\\data\\ task1.csv'

In [ ]:
salinity_df.head()

,District Name,salinity_percent,districtname
0,South Andamans,0.000000,south andamans
1,Nicobars,0.000000,nicobars
2,North And Middle Andaman,0.000000,north and middle andaman
3,Anantapur,12.341882,anantapur
4,Chittoor,2.652205,chittoor


In [ ]:
salinity_df['districtname']= salinity_df['District Name'].str.lower().str.strip()

In [ ]:
sugarcane_df['districtname'] = sugarcane_df['districtname'].str.lower().str.strip()

In [ ]:
sugarcane_df = pd.merge(
    sugarcane_df,
    salinity_df[['districtname', 'Average Salinity/Alkalinity (Percentage)']],  # adjust if column name is different
    on='districtname',
    how='inner'
)

In [ ]:
print(sugarcane_df[['districtname', 'Average Salinity/Alkalinity (Percentage)']].dropna().head())


  districtname  Average Salinity/Alkalinity (Percentage)
0         durg                                  0.000000
1       bastar                                  0.000000
2       raipur                                  0.000000
3     bilaspur                                  0.390813
4      raigarh                                  0.000000


In [ ]:
print(sugarcane_df['Average Salinity/Alkalinity (Percentage)'].notnull().sum(), "districts with salinity data")


168 districts with salinity data


In [ ]:
# Save the DataFrame after merging salinity data
sugarcane_df.to_csv('D:\ECO221_Group-25\data\ task2.csv', index=False)

<>:2: SyntaxWarning: invalid escape sequence '\E'
<>:2: SyntaxWarning: invalid escape sequence '\E'
C:\Users\DELL\AppData\Local\Temp\ipykernel_20032\4240613749.py:2: SyntaxWarning: invalid escape sequence '\E'
  sugarcane_df.to_csv('D:\ECO221_Group-25\data\ task2.csv', index=False)
C:\Users\DELL\AppData\Local\Temp\ipykernel_20032\4240613749.py:2: SyntaxWarning: invalid escape sequence '\E'
  sugarcane_df.to_csv('D:\ECO221_Group-25\data\ task2.csv', index=False)


PermissionError: [Errno 13] Permission denied: 'D:\\ECO221_Group-25\\data\\ task2.csv'

In [ ]:
#Question 2
import statsmodels.formula.api as smf


In [ ]:
df_cb = sugarcane_df.copy()

In [ ]:
df_cb['irrigatedarea_new'] = df_cb['irrigatedarea1000hectares'] + 1

In [ ]:
df_cb['unirrigated_area'] = df_cb['area1000hectares'] - df_cb['irrigatedarea_new']

In [338]:
df_cb = df_cb.rename(columns={'Average Salinity/Alkalinity (Percentage)': 'salinity_percent'})

In [339]:
required_cols = [
    "production1000tonnes", "area1000hectares", "irrigatedarea_new", "unirrigated_area",
    "nitrogenconsumptiontonnes", "phosphateconsumptiontonnes", "potashconsumptiontonnes",
    "annual_rainfall_mm", "salinity_percent"
]
df_cb = df_cb.dropna(subset=required_cols)

In [340]:
# Remove non-positive values (can't take log of 0 or negative)
for col in ["production1000tonnes", "area1000hectares", "irrigatedarea_new", "unirrigated_area",
            "nitrogenconsumptiontonnes", "phosphateconsumptiontonnes", "potashconsumptiontonnes"]:
    df_cb = df_cb[df_cb[col] > 0]

In [341]:
# Create log-transformed variables
df_cb["log_production"] = np.log(df_cb["production1000tonnes"])
df_cb["log_area"] = np.log(df_cb["area1000hectares"])
df_cb["log_irrigated"] = np.log(df_cb["irrigatedarea_new"])
df_cb["log_unirrigated"] = np.log(df_cb["unirrigated_area"] + 1e-4)  # small constant to avoid log(0)
df_cb["log_nitrogen"] = np.log(df_cb["nitrogenconsumptiontonnes"])
df_cb["log_phosphate"] = np.log(df_cb["phosphateconsumptiontonnes"])
df_cb["log_potash"] = np.log(df_cb["potashconsumptiontonnes"])

In [342]:
# Run Cobb-Douglas OLS regression
formula = (
    "log_production ~ log_area + log_irrigated + log_unirrigated + "
    "log_nitrogen + log_phosphate + log_potash + "
    "annual_rainfall_mm + salinity_percent"
)

In [343]:
model = smf.ols(formula=formula, data=df_cb).fit()


In [344]:
print(df_cb.columns)

Index(['crop', 'area1000hectares', 'production1000tonnes',
       'irrigatedarea1000hectares', 'districtcode', 'year', 'statecode',
       'statename', 'nitrogenconsumptiontonnes', 'phosphateconsumptiontonnes',
       'potashconsumptiontonnes', 'districtname', 'irrigatedarea_capped',
       'annual_rainfall_mm', 'monsoon_rainfall_mm', 'salinity_percent',
       'irrigatedarea_new', 'unirrigated_area', 'log_production', 'log_area',
       'log_irrigated', 'log_unirrigated', 'log_nitrogen', 'log_phosphate',
       'log_potash'],
      dtype='object')


In [345]:
formula = 'log_production ~ log_area + log_irrigated + log_unirrigated + log_nitrogen + log_phosphate + log_potash + annual_rainfall_mm + salinity_percent'

In [ ]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         log_production   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     71.24
Date:                Sat, 19 Apr 2025   Prob (F-statistic):           5.54e-23
Time:                        18:10:43   Log-Likelihood:                -37.947
No. Observations:                  54   AIC:                             93.89
Df Residuals:                      45   BIC:                             111.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.2157      1